# ROS Service

- Service = Another way that nodes can communicate with each other. Services allow nodes to send a request and receive a response.

- Service = Kolejny sposób komunikacji. Zaletą serwisów jest możliwość wysłania zapytania do serwera. Podobnie jak dla ROS topic należy znać format wiadomości. Serwisy w danej paczce ROS przechowywane są w katalogu srv, a rozszerzenie serwisu to **.srv**. 

# 1) Struktura wiadomości serwisowej

Wyróżnia się podział na format wiadomości:
- wysyłany przez klienta (górna część wiadomości nad znakiem **---**)
- odpowiedź serwera (dolna część wiadomości pod znakiem **---**)

In [51]:
!rossrv info pkg_tsr/Task #jest udostepnione w pkg_tsr taki format, ze bedzie przyjmowal dwa floaty i zwracal string

float64 x
float64 y
---
string result



Z lewej strony należy podać typ wiadomości ROS. Mogą być one bardziej złożone i składać się z już utworzonych
wiadomości (ROS msg). Z prawej strony podawana jest nazwa pola.

Typ serwisu to **nazwa paczki + nazwa serwisu.srv**. Wyświetlenie przykładowego serwisu znajdującego się w paczce tsr_materials:

# 2) Wyświetlanie listy aktualnie dostępnych serwisów

In [2]:
!rosservice list

/Talker/get_loggers
/Talker/set_logger_level
/clear
/kill
/reset
/rosout/get_loggers
/rosout/set_logger_level
/spawn
/turtle1/set_pen
/turtle1/teleport_absolute
/turtle1/teleport_relative
/turtlesim/get_loggers
/turtlesim/set_logger_level


# 3) Wyświetlanie dostępnej pomocy dla polecenia rosservice.

In [3]:
!rosservice --help

Commands:
	rosservice args	print service arguments
	rosservice call	call the service with the provided args
	rosservice find	find services by service type
	rosservice info	print information about service
	rosservice list	list active services
	rosservice type	print service type
	rosservice uri	print service ROSRPC uri

Type rosservice <command> -h for more detailed usage, e.g. 'rosservice call -h'



### Dostępne serwisy dla turtlesim node

W podstawowym zakresie dostępne są następujące serwisy bez względu na robota:
- /clear - wyczyszczenie narysowanych ścieżek
- /kill - usunięcie robota
- /reset - reset środowiska do stanu początkowego
- /spawn - utworzenie nowego robota

Dla pojedynczego utworzonego robota w przestrzeni nazw na przykładzie **turtle1** dostępne są nastpujące serwisy:
- /turtle1/set_pen - ustawienie koloru pędzla do rysowania
- /turtle1/teleport_absolute - natychmiastowe przeniesienie robota do wskazanej lokalizacji
- /turtle1/teleport_relative - przeniesienie robota, współrzędne podawane w układzie robota

# 4) Rosservice info

In [5]:
!rosservice info clear

Node: /turtlesim
URI: rosrpc://localhost:33161
Type: std_srvs/Empty
Args: 


In [8]:
!rosservice info turtle1/teleport_absolute

Node: /turtlesim
URI: rosrpc://localhost:40793
Type: turtlesim/TeleportAbsolute
Args: x y theta


In [21]:
!rosservice info kill

Node: /turtlesim
URI: rosrpc://localhost:58493
Type: turtlesim/Kill
Args: name


# 5) Rosservice call
- rosservice call [service] [args]

In [37]:
!rosservice call spawn 5 5 0 Delmiak #x,y,theta,nazwa #x,y <0,11>

name: "Delmiak"


In [13]:
!rosservice call clear #czyszczenie ścieżek

- Kwadrat /*nazwa_robota*/teleport_absolute

In [38]:
#rosservice call [service]                  [args]
!rosservice call /Delmiak/teleport_absolute 10 1 0
!rosservice call /Delmiak/teleport_absolute 10 10 0
!rosservice call /Delmiak/teleport_absolute 1 10 0
!rosservice call /Delmiak/teleport_absolute 1 1 0

- Usuwa robota

In [10]:
!rosservice call /kill Delmiak

- Usuwa wszystkie roboty

In [89]:
!rosservice call reset #usuwa żółwie, utworzone linie i tworzy nowego żółwia

# 6) ServiceProxy - Klient

- rospy.ServiceProxy(name, service_class, persistent=False, headers=None)

In [15]:
import rospy
rospy.init_node("serwis_node_test")

- Empty:

In [16]:
#import serwisu
from std_srvs.srv import Empty

In [17]:
clear_map = rospy.ServiceProxy('clear', Empty)
clear_map()

In [18]:
reset_sim_state = rospy.ServiceProxy('reset', Empty)
reset_sim_state()

- Spawn:

In [61]:
from turtlesim.srv import Spawn

In [62]:
create_new_robot = rospy.ServiceProxy('spawn', Spawn)
imie = create_new_robot(3, 3, 0, "Delmiak")

In [17]:
imie.name

'NewTurtle'

- A dictionary or other mapping object used to store an object's (writable) attributes

In [21]:
sp=Spawn
sp.__dict__

mappingproxy({'__module__': 'turtlesim.srv._Spawn',
              '_type': 'turtlesim/Spawn',
              '_md5sum': '0b2d2e872a8e2887d5ed626f2bf2c561',
              '_request_class': turtlesim.srv._Spawn.SpawnRequest,
              '_response_class': turtlesim.srv._Spawn.SpawnResponse,
              '__dict__': <attribute '__dict__' of 'Spawn' objects>,
              '__weakref__': <attribute '__weakref__' of 'Spawn' objects>,
              '__doc__': None})

#### Zadanie 5
Ustawić kolor pędzla na czerwony dla nowo utworzonego robota używając kodu w Pythonie

In [63]:
from turtlesim.srv import SetPen
set_pen = rospy.ServiceProxy('/Delmiak/set_pen', SetPen)
set_pen(255, 0, 0, 3, 0)

#### Zadanie 6
Tak jak wcześniej narysować kwadrat, ale wykorzystując program w Pythonie.

In [64]:
# UZUPEŁNIĆ
from turtlesim.srv import TeleportAbsolute
move_tu = rospy.ServiceProxy('/Delmiak/teleport_absolute',TeleportAbsolute)

move_tu(3,1,0)
move_tu(3,3,0)
move_tu(1,3,0)
move_tu(1,1,0)

# 7) Service

- rospy.Service(name, service_class, handler, buff_size=65536)

In [1]:
from pkg_tsr.srv import Task
from turtlesim.srv import TeleportAbsolute
from std_srvs.srv import Empty

In [25]:
def draw_square_function(req):
    width = 5
    print("Init pose x=%s, y=%s" % (req.x, req.y))
    
    #ServiceProxy
    set_pose = rospy.ServiceProxy('/turtle1/teleport_absolute', TeleportAbsolute)
    clear_map = rospy.ServiceProxy('clear', Empty)
    
    #SerivceProxy_Use
    set_pose(req.x,req.y,0)
    clear_map()
    
    #Set pose
    set_pose(req.x,req.y + width,0)
    set_pose(req.x + width,req.y + width,0)
    set_pose(req.x + width, req.y,0)
    set_pose(req.x, req.y,0)
    
    return "finished"

In [26]:
#rospy.Service(name, service_class, handler, buff_size=65536)
s = rospy.Service('robot_teleport', Task, draw_square_function)

In [27]:
draw_square = rospy.ServiceProxy('robot_teleport', Task)
resp = draw_square(1, 2)

Init pose x=1.0, y=2.0


 - Odczyt otrzymanej wartości z serwera:

In [ ]:
resp.result

#### UWAGA
Wyłączenie serwisu - gdy wprowadzone zostaną jakieś zmiany w funkcji serwisowej, aby nie restartować Kernela Jupyter Notebook, można poniższą metodą shutdown() zatrzymać działający serwis.

In [ ]:
s.shutdown()

#### Zadanie 7
#### 7.1 
- **go_to_goal()** dojazd do punktu o podanych współrzędnych.
- W klasie Robot znajduje się serwis odpowiedzialny za wywołanie tej metody.
- Serwis korzysta z typu **tsr_materials/Task.srv**.
- W zapytaniu od klienta w polach x,y należy przesłać współrzędne celu.
- W odpowiedzi na zapytanie klienta należy odesłać informację o zakończeniu działania.
- Metoda powinna wysyłać odpowiednie prędkości sterujące robotem (postępową i obrotową)na topicu **/nazwa_robot/cmd_vel**.
- Należy ustawić wartość zmiennej od tolerancji dojazdu do celu (np.0.05) i skorzystać z niej przy dojeżdżaniu do celu.
- Dodatkowo powinny zostać uwzględnione ograniczenia prędkości (np. 0.3). 

#### 7.2 
- Analogicznie do metody **go_to_goal** dołożyć inną metodę umożliwiającą dojazd do wskazanego punktu i rozpoczęcie ruchu po dowolnej krzywej np. łamana, spirala, okrąg.

In [2]:
import rospy
from turtlesim.srv import Spawn
from turtlesim.msg import Pose
from geometry_msgs.msg import Twist
from pkg_tsr.srv import Task
import math

rospy.init_node("serwis_node_test")

### 7.1

In [3]:
class Robot:
    def __init__(self, name, x, y):
        self.name = name
        self.pose = Pose()
        create_new_turtle = rospy.ServiceProxy('spawn', Spawn)
        create_new_turtle(x, y, 0, name)
        
        self.velocity_pub = rospy.Publisher(name+'/cmd_vel', Twist, queue_size=10)
        self.pose_sub = rospy.Subscriber(name+'/pose',Pose, self.callback_pose)
        self.follow_service = rospy.Service(name+'/go_to_goal', Task, self.go_to_goal)
        
    def go_to_goal(self, msg_srv):
        max_speed = 0.3
        omega=0.3
        tolerance = 0.05
        
        #polozenie celu
        x_c=msg_srv.x
        y_c=msg_srv.y
        
        msg_vel=Twist()
        
        while True:
            #polozenie robota
            x=self.pose.x
            y=self.pose.y
            theta=self.pose.theta
            
            msg_vel.linear.x=max_speed
            if math.atan2(y_c-y,x_c-x)!=theta:
                if theta<math.atan2(y_c-y,x_c-x):
                    msg_vel.angular.z=omega
                else:
                    msg_vel.angular.z=-omega
            else:
                msg_vel.angular.z=0
            if math.sqrt((x_c-x)**2+(y_c-y)**2)<tolerance: 
                return "finished"
            self.velocity_pub.publish(msg_vel)
            
    def go_to_goal_and_do_circle(self, msg_srv):
        max_speed = 0.3
        omega=0.3
        tolerance = 0.05
        
        go_to_goal=True
        do_circle = False
        
        #polozenie celu
        x_c=msg_srv.x
        y_c=msg_srv.y
        
        msg_vel=Twist()
        
        while True:
            #polozenie robota
            x=self.pose.x
            y=self.pose.y
            theta=self.pose.theta
            
            if go_to_goal:
                msg_vel.linear.x=max_speed
                if math.atan2(y_c-y,x_c-x)!=theta:
                    if theta<math.atan2(y_c-y,x_c-x):
                        msg_vel.angular.z=omega
                    else:
                        msg_vel.angular.z=-omega
                else:
                    msg_vel.angular.z=0
                if math.sqrt((x_c-x)**2+(y_c-y)**2)<tolerance: 
                    go_to_goal=False
                    do_circle=True
            if do_circle:
                msg_vel.angular.z=omega
            self.velocity_pub.publish(msg_vel)
    
    def callback_pose(self, pose_msg):
        self.pose=pose_msg
    def unregister(self):
        self.follow_service.shutdown()

In [31]:
nowy_robot.unregister()

In [32]:
!rosservice call reset

In [4]:
nowy_robot = Robot("Turtle",4,4)

In [5]:
nowy_robot.pose

x: 4.0
y: 4.0
theta: 0.0
linear_velocity: 0.0
angular_velocity: 0.0

- Utworzyć klienta i zlecić zadanie dojazdu do kilku różnych punktów:

In [6]:
def go_to_different_goals(nowy_robot):
    go_to_goal_SP=rospy.ServiceProxy(nowy_robot.name+'/go_to_goal', Task)
    go_to_goal_SP(8,7)
    go_to_goal_SP(2,3)
    go_to_goal_SP(4,6)

In [7]:
go_to_different_goals(nowy_robot)

In [ ]:
nowy_robot.pose

### 7.2 
- Analogicznie do metody go_to_goal dołożyć inną metodę umożliwiającą dojazd do wskazanego punktu i rozpoczęcie ruchu po dowolnej krzywej np. łamana, spirala, okrąg.

In [23]:
srv = Task()
srv.x=8
srv.y=7
nowy_robot.go_to_goal_and_do_circle(srv)

KeyboardInterrupt: 

## Dodatek

Tworzenie własnych wiadomości serwisowych. W paczce znajduje się katalog srv w którym można utworzyć dodatkowe wiadomości serwisowe. Konfiguracja w przedstawionym dodatku ogranicza się do używania typów wiadomości z poniższego linku:

http://docs.ros.org/en/melodic/api/std_msgs/html/index-msg.html

In [ ]:
!rossrv show pkg_tsr/Task

In [ ]:
!rosmsg show std_msgs/ColorRGBA

Przykładowo rozbudowany serwis o wiadomość typu ColorRGBA może wyglądać następująco. 

Dodanie nowej wiadomości wymaga zmian w plikach konfiguracyjnych paczki. Ze względu na przyjęte uproszczenie w 
konfiguracji w paczce tsr_materials w pliku **CMakeLists.txt** w miejscu (od 57 linii):

Należy dopisać nazwę swojego serwisu jak powyżej. Zamiast **Nowy.srv** należy podać nazwę serwisu pod jakim został on zapisany w katalogu srv.

In [ ]:
!rossrv show pkg_tsr/... #uzupelnic nazwa nowego serwisu, jesli wszystko zostalo prawidlowo utworzone 
# powinna pojawic sie jego struktura